# Import Libraries

In [92]:
import numpy as np
import ee 
# from ee_plugin import Map 
import geemap
import pprint as pprint
from datetime import datetime


In [93]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# Import features collections and data

In [94]:
Map = geemap.Map(center=(40, -100), zoom=4)
geometry = ee.FeatureCollection('users/2014ee070/YoloCountyBoundary')
data = ee.FeatureCollection('users/2014ee070/yoloCDWR2016')
yolo = geometry

# Defining inputs

In [95]:
start_date = '2016-03-01'
end_date = '2016-11-30'
nClasses = 43
num_classes = 43

In [96]:
classes = ["Rice", "Alfalfa and Alfalfa Mixtures", "Wheat", "Sunflowers", "Safflower", "Corn, Sorghum and Sudan",
           "Young Perennials", "Melons, Squash and Cucumbers", "Tomatoes", "Carrots", "Onions and Garlic", 
           "Potatoes and Sweet Potatoes", "Strawberries", "Bush Berries", "Cherries", "Beans (Dry)", "Peppers",
           "Cole Crops", "Miscellaneous Truck Crops", "Miscellaneous Grain and Hay", "Miscellaneous Field Crops", 
           "Lettuce/Leafy Greens", "Olives", "Plums, Prunes and Apricots", "Pistachios", "Almonds", "Walnuts", "Grapes", 
           "Pears", "Apples", "Citrus", "Miscellaneous Subtropical Fruits", "Peaches/Nectarines", "Pomegranates", 
           "Dates", "Flowers, Nursery and Christmas Tree Farms", "Miscellaneous Grasses", "Miscellaneous Deciduous", 
           "Mixed Pasture", "Greenhouse", "Managed Wetland", "Idle", "Urban"]



# Defining Functions

In [112]:
def maskL8sr(image):
    # Bit 4: Cloud
    # Bit 5,6: Cloud confidence
    # Bit 7,8: Cloud Shadow
    cloudShadowBitMask = 1 << 8
    cloudConfidenceBitMask = 1 << 6
    cloudsBitMask = 1 << 4

    # Get the pixel QA band.
    qa = image.select('BQA')

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudConfidenceBitMask).eq(0)) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))

    # Return the masked image, scaled to reflectance, without the QA bands.
    return image.updateMask(mask) \
      .select("B[2-7]*").divide(10000) \
      .copyProperties(image, ["system:time_start"])


def addNDVI(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)


def addGNDVI(image):
    gndvi = image.normalizedDifference(['B5', 'B3']).rename('GNDVI')
    return image.addBands(gndvi)


def addEVI(image):
    evi = image.expression("2.5 * ((nir-red)/(nir + 6*red -7.5*blue + 1))",
    {
    'nir' : image.select("B5"),
    'red' : image.select("B4"),
    'blue': image.select("B2")
    }).rename('EVI')
    return image.addBands(evi)


def addSAVI(image):
    savi = image.expression("1.5*(red-green)/(red+green+0.5)",
    {
    'red' : image.select("B4"),
    'green': image.select("B3")
    }).rename('SAVI')
    return image.addBands(savi)


def addNDWI(image):
    ndwi = image.normalizedDifference(['B3', 'B5']).rename('NDWI')
    return image.addBands(ndwi)

def addNDBI(image):
    ndbi = image.normalizedDifference(['B6', 'B5']).rename('NDBI')
    return image.addBands(ndbi)

def addBSI(image):
    bsi = image.expression("((swir + red) - (nir + blue))/((swir + red) + (nir + blue))",
    {
    'blue' : image.select("B2"),
    'red' : image.select("B4"),
    'nir' : image.select("B5"),
    'swir': image.select("B6")
    }).rename('BSI')
    bsi = bsi.divide(10000)
    return image.addBands(bsi) 

def ymdList(imgcol):
    def iter_func(image, newlist):
        date = ee.String(image.date().format("YYYY-MM-dd"))
        newlist = ee.List(newlist)
        return ee.List(newlist.add(date).sort())

    return imgcol.iterate(iter_func, ee.List([]))

def temporalCollection(collection, start, count, interval, units):
    sequence = ee.List.sequence(0, ee.Number(count).subtract(1))
    originalStartDate = ee.Date(start)

    def func_aoa(i):
        startDate = originalStartDate.advance(ee.Number(interval).multiply(i), units)
        endDate = originalStartDate.advance(ee.Number(interval).multiply(ee.Number(i).add(1)), units)
        resultImage = collection.filterDate(startDate, endDate).median() \
                          .set('system:time_start', startDate.millis()) \
                          .set('system:time_end', endDate.millis())
        return resultImage

    return ee.ImageCollection(sequence.map(func_aoa))


# Preprocessing Imagery Landsat 8

In [113]:
Map.centerObject(geometry)
l8Collection = ee.ImageCollection("LANDSAT/LC08/C01/T1") \
                  .filterDate(start_date, end_date) \
                  .filterBounds(geometry) \
                  .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
                  .map(maskL8sr) \
                  .map(addNDVI).map(addGNDVI).map(addEVI).map(addSAVI).map(addBSI).map(addNDWI).map(addNDBI) \
                  .sort('system:time_start', True)
# ['B2', 'B3', 'B4', 'B5', 'B6','B7', 'NDVI', 'GNDVI', 'EVI', 'SAVI', 'BSI', 'NDWI', 'NDBI']
print("images In collection",l8Collection.size().getInfo())
l8Collection = temporalCollection(l8Collection, start_date, 9, 30, 'day')

images In collection 19


## Displaying all images selected each month

In [114]:
# # #*************** Get 12 images from L8 ***********************#

l8ImagesList = l8Collection.toList(l8Collection.size())
finalImage = ee.Image(l8ImagesList.get(0)).clip(geometry)
ts = ee.Image(l8ImagesList.get(0)).get('system:time_start').getInfo()
print(datetime.utcfromtimestamp(ts/1000).strftime('%Y-%m-%d %H:%M:%S'))

# Map.addLayer(finalImage, {'bands': ['B4', 'B3', 'B2'], 'min':6000,'max': 12000}, 'L8-1')

n = 1
i = 1
while i < 9:
    n = n + 1
    current = ee.Image(l8ImagesList.get(i)).clip(geometry)

    if(i == 1):
        currentDate = ee.Date(current.get('system:time_start'))
        meanImage = l8Collection.filterDate(
                    currentDate.advance(-1, 'month'), currentDate.advance(1, 'month')).mean() \
                    .clip(yolo) \
                    .set('system:time_start', currentDate.millis())
        current = meanImage.where(current, current)

    ts = current.get('system:time_start').getInfo()
    print(datetime.utcfromtimestamp(ts/1000).strftime('%Y-%m-%d %H:%M:%S'))
    
    finalImage = finalImage.addBands(current)
    # Map.addLayer(current, {'bands': ['B4', 'B3', 'B2'], 'min':6000,'max': 12000}, 'L8-'+n)
    i = i + 1

image = finalImage.clip(geometry).toFloat()
featuresNames = image.bandNames()
print('All features', featuresNames.getInfo())


2016-03-01 00:00:00
2016-03-31 00:00:00
2016-04-30 00:00:00
2016-05-30 00:00:00
2016-06-29 00:00:00
2016-07-29 00:00:00
2016-08-28 00:00:00
2016-09-27 00:00:00
2016-10-27 00:00:00
All features ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'NDVI', 'GNDVI', 'EVI', 'SAVI', 'BSI', 'NDWI', 'NDBI', 'B2_1', 'B3_1', 'B4_1', 'B5_1', 'B6_1', 'B7_1', 'NDVI_1', 'GNDVI_1', 'EVI_1', 'SAVI_1', 'BSI_1', 'NDWI_1', 'NDBI_1', 'B2_2', 'B3_2', 'B4_2', 'B5_2', 'B6_2', 'B7_2', 'NDVI_2', 'GNDVI_2', 'EVI_2', 'SAVI_2', 'BSI_2', 'NDWI_2', 'NDBI_2', 'B2_3', 'B3_3', 'B4_3', 'B5_3', 'B6_3', 'B7_3', 'NDVI_3', 'GNDVI_3', 'EVI_3', 'SAVI_3', 'BSI_3', 'NDWI_3', 'NDBI_3', 'B2_4', 'B3_4', 'B4_4', 'B5_4', 'B6_4', 'B7_4', 'NDVI_4', 'GNDVI_4', 'EVI_4', 'SAVI_4', 'BSI_4', 'NDWI_4', 'NDBI_4', 'B2_5', 'B3_5', 'B4_5', 'B5_5', 'B6_5', 'B7_5', 'NDVI_5', 'GNDVI_5', 'EVI_5', 'SAVI_5', 'BSI_5', 'NDWI_5', 'NDBI_5', 'B2_6', 'B3_6', 'B4_6', 'B5_6', 'B6_6', 'B7_6', 'NDVI_6', 'GNDVI_6', 'EVI_6', 'SAVI_6', 'BSI_6', 'NDWI_6', 'NDBI_6', 'B2_7', 'B3_7

# FIltering and remapping CRS data labels

In [115]:
#************* remapping crops to major crops *****************#

classSizes = {}
for i in range(0, nClasses, 1):
    classSizes[classes[i]] = data.filter(ee.Filter.eq('landcover', i)).size().getInfo()

pprint.pprint(classes)
pprint.pprint(classSizes)

['Rice',
 'Alfalfa and Alfalfa Mixtures',
 'Wheat',
 'Sunflowers',
 'Safflower',
 'Corn, Sorghum and Sudan',
 'Young Perennials',
 'Melons, Squash and Cucumbers',
 'Tomatoes',
 'Carrots',
 'Onions and Garlic',
 'Potatoes and Sweet Potatoes',
 'Strawberries',
 'Bush Berries',
 'Cherries',
 'Beans (Dry)',
 'Peppers',
 'Cole Crops',
 'Miscellaneous Truck Crops',
 'Miscellaneous Grain and Hay',
 'Miscellaneous Field Crops',
 'Lettuce/Leafy Greens',
 'Olives',
 'Plums, Prunes and Apricots',
 'Pistachios',
 'Almonds',
 'Walnuts',
 'Grapes',
 'Pears',
 'Apples',
 'Citrus',
 'Miscellaneous Subtropical Fruits',
 'Peaches/Nectarines',
 'Pomegranates',
 'Dates',
 'Flowers, Nursery and Christmas Tree Farms',
 'Miscellaneous Grasses',
 'Miscellaneous Deciduous',
 'Mixed Pasture',
 'Greenhouse',
 'Managed Wetland',
 'Idle',
 'Urban']
{'Alfalfa and Alfalfa Mixtures': 575,
 'Almonds': 753,
 'Apples': 13,
 'Beans (Dry)': 26,
 'Bush Berries': 4,
 'Carrots': 9,
 'Cherries': 5,
 'Citrus': 32,
 'Cole Crops

## Splitting data and making image

In [116]:
yolo_data = data.randomColumn('random', 2016)
training = yolo_data.filter(ee.Filter.lt('random', 0.6));
validation = yolo_data.filter(ee.Filter.And(
                      ee.Filter.gte('random', 0.6), 
                      ee.Filter.lt('random', 0.8)));
testing = yolo_data.filter(ee.Filter.gte('random', 0.8));
print(training.size().getInfo(), validation.size().getInfo(), testing.size().getInfo())

training_label = training.reduceToImage(
    properties =  ['landcover'],
    reducer = ee.Reducer.first()
)
val_label = validation.reduceToImage(
    properties =  ['landcover'],
    reducer = ee.Reducer.first()
)
testing_label = testing.reduceToImage(
    properties =  ['landcover'],
    reducer = ee.Reducer.first()
)



4408 1467 1489


In [117]:
a = validation.filter(ee.Filter.eq('MULTIUSE', 'S'))
print(a.size().getInfo())

1432


In [118]:
a = testing.filter(ee.Filter.eq('MULTIUSE', 'S'))
print(a.size().getInfo())

1437


In [129]:
Map.addLayer(training_label, {'palette': 'FF0000'}, 'Training')
Map.addLayer(val_label, {'palette': '0000FF'}, 'validation')
Map.addLayer(testing_label, {'palette': '0000FF'}, 'Testing')
Map.centerObject(training, 10)
Map.addLayer(patch2)
Map

Map(center=[38.66289070025362, -121.81103976701577], controls=(WidgetControl(options=['position'], widget=HBox…

In [82]:
# old_labels = [0, 1, 2, 3, 4, 5, 6, 8, 19, 27, 
#               14, 23, 24, 25, 26, 28, 29, 32, 33, 37, 
#               22, 30, 31, 34,
#               36, 38, 
#               7, 9, 10, 11, 12, 13, 15, 16, 17, 18, 20, 21, 35, 39, 
#               40, 41, 42]
# new_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
#               10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
#               11, 11, 11, 11, 
#               12, 12, 
#               13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 
#               14, 15, 16]
# training = training.remap(
#   lookupIn = old_labels,
#   lookupOut = new_labels,
#   columnName = 'landcover',
# )

# validation = validation.remap(
#   lookupIn = old_labels,
#   lookupOut = new_labels,
#   columnName = 'landcover',
# )

# testing = testing.remap(
#   lookupIn = old_labels,
#   lookupOut = new_labels,
#   columnName = 'landcover',
# )
# #************* converting shapefile vector to raster ****************#


# # label = label.remap(old_labels,
# #   to = new_labels)

In [119]:
label = yolo_data.reduceToImage(
    properties =  ['landcover'],
    reducer = ee.Reducer.first()
)
label = label.select('first').rename('landcover')


## Making final Image

In [120]:
kernel_size = 3
image = finalImage.addBands(label).toFloat()

neighbourhood_image = image.neighborhoodToArray(ee.Kernel.rectangle(kernel_size, kernel_size, 'pixels'))
# Map.addLayer(neighbourhood_image, {'bands': ['B4', 'B3', 'B2'], 'min': 6000, 'max': 12000}, 'L8_extended')
Map.addLayer(neighbourhood_image, {'bands': ['B4', 'B3', 'B2'], 'min': 0.6, 'max': 1.2}, 'L8_extended')

Map.centerObject(yolo)
# Map

In [85]:
neighbourhood_image.bandNames().getInfo()

['B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'NDWI',
 'NDBI',
 'NDVI',
 'GNDVI',
 'EVI',
 'SAVI',
 'BSI',
 'B2_1',
 'B3_1',
 'B4_1',
 'B5_1',
 'B6_1',
 'B7_1',
 'NDWI_1',
 'NDBI_1',
 'NDVI_1',
 'GNDVI_1',
 'EVI_1',
 'SAVI_1',
 'BSI_1',
 'B2_2',
 'B3_2',
 'B4_2',
 'B5_2',
 'B6_2',
 'B7_2',
 'NDWI_2',
 'NDBI_2',
 'NDVI_2',
 'GNDVI_2',
 'EVI_2',
 'SAVI_2',
 'BSI_2',
 'B2_3',
 'B3_3',
 'B4_3',
 'B5_3',
 'B6_3',
 'B7_3',
 'NDWI_3',
 'NDBI_3',
 'NDVI_3',
 'GNDVI_3',
 'EVI_3',
 'SAVI_3',
 'BSI_3',
 'B2_4',
 'B3_4',
 'B4_4',
 'B5_4',
 'B6_4',
 'B7_4',
 'NDWI_4',
 'NDBI_4',
 'NDVI_4',
 'GNDVI_4',
 'EVI_4',
 'SAVI_4',
 'BSI_4',
 'B2_5',
 'B3_5',
 'B4_5',
 'B5_5',
 'B6_5',
 'B7_5',
 'NDWI_5',
 'NDBI_5',
 'NDVI_5',
 'GNDVI_5',
 'EVI_5',
 'SAVI_5',
 'BSI_5',
 'B2_6',
 'B3_6',
 'B4_6',
 'B5_6',
 'B6_6',
 'B7_6',
 'NDWI_6',
 'NDBI_6',
 'NDVI_6',
 'GNDVI_6',
 'EVI_6',
 'SAVI_6',
 'BSI_6',
 'B2_7',
 'B3_7',
 'B4_7',
 'B5_7',
 'B6_7',
 'B7_7',
 'NDWI_7',
 'NDBI_7',
 'NDVI_7',
 'GNDVI_7',
 'EVI_7',
 'SAVI_

In [86]:
image.bandNames().getInfo()

['B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'NDWI',
 'NDBI',
 'NDVI',
 'GNDVI',
 'EVI',
 'SAVI',
 'BSI',
 'B2_1',
 'B3_1',
 'B4_1',
 'B5_1',
 'B6_1',
 'B7_1',
 'NDWI_1',
 'NDBI_1',
 'NDVI_1',
 'GNDVI_1',
 'EVI_1',
 'SAVI_1',
 'BSI_1',
 'B2_2',
 'B3_2',
 'B4_2',
 'B5_2',
 'B6_2',
 'B7_2',
 'NDWI_2',
 'NDBI_2',
 'NDVI_2',
 'GNDVI_2',
 'EVI_2',
 'SAVI_2',
 'BSI_2',
 'B2_3',
 'B3_3',
 'B4_3',
 'B5_3',
 'B6_3',
 'B7_3',
 'NDWI_3',
 'NDBI_3',
 'NDVI_3',
 'GNDVI_3',
 'EVI_3',
 'SAVI_3',
 'BSI_3',
 'B2_4',
 'B3_4',
 'B4_4',
 'B5_4',
 'B6_4',
 'B7_4',
 'NDWI_4',
 'NDBI_4',
 'NDVI_4',
 'GNDVI_4',
 'EVI_4',
 'SAVI_4',
 'BSI_4',
 'B2_5',
 'B3_5',
 'B4_5',
 'B5_5',
 'B6_5',
 'B7_5',
 'NDWI_5',
 'NDBI_5',
 'NDVI_5',
 'GNDVI_5',
 'EVI_5',
 'SAVI_5',
 'BSI_5',
 'B2_6',
 'B3_6',
 'B4_6',
 'B5_6',
 'B6_6',
 'B7_6',
 'NDWI_6',
 'NDBI_6',
 'NDVI_6',
 'GNDVI_6',
 'EVI_6',
 'SAVI_6',
 'BSI_6',
 'B2_7',
 'B3_7',
 'B4_7',
 'B5_7',
 'B6_7',
 'B7_7',
 'NDWI_7',
 'NDBI_7',
 'NDVI_7',
 'GNDVI_7',
 'EVI_7',
 'SAVI_

# Sampling

In [121]:
# sampling only points for now
training_points = image.select('landcover')\
          .sampleRegions(
            collection= training,
            scale=80,
            tileScale =1,
            properties= ['landcover'],
            geometries = True
          )
val_points = image.select('landcover')\
          .sampleRegions(
            collection= validation,
            scale=80,
            tileScale = 1,
            properties= ['landcover'],
            geometries = True
          )

testing_points = image.select('landcover')\
          .sampleRegions(
            collection= testing,
            scale=80,
            tileScale = 1,
            properties= ['landcover'],
            geometries = True
          )


In [88]:
# # get lat longs of sampled points
# training_pts = training_points.geometry().getInfo()['coordinates']
# val_pts = val_points.geometry().getInfo()['coordinates']
# testing_pts = testing_points.geometry().getInfo()['coordinates']


In [89]:
len(training_pts), len(val_pts), len(testing_pts)

(156470, 59728, 69150)

In [128]:
patch2 = ee.Geometry.Polygon([[-121.98298540330632,38.589919143886206],
[-121.77287187791569,38.589919143886206],
[-121.77287187791569,38.68110064858854],
[-121.98298540330632,38.68110064858854],
[-121.98298540330632,38.589919143886206]])
Map.addLayer(patch2)

In [ ]:
patch2 = yolo

In [126]:
Task = ee.batch.Export.image.toDrive(
    image = image.clip(patch2),
    description = 'patch of yolo',
    folder = 'Yolo_2016_patch',
    fileNamePrefix = 'Yolo_2016_patch5',
    scale = 30,
    fileFormat = 'GeoTIFF',
    region = patch2
)
Task.start()

In [124]:
patch2.area(0.01).divide(1e6).getInfo()

59.150368959482066

In [39]:
# testing_features = testing.toList(1489)
# for i in range(1489):
#     feature = ee.Feature(testing_features.get(i))
#     fcp = testing_points.filterBounds(feature.geometry())
#     train_db = neighbourhood_image.sampleRegions(collection=fcp, scale=30)
#     Task = ee.batch.Export.table.toDrive(
#         collection=train_db,        
#         description="testing_yolo_2016"+str(i),
#         fileNamePrefix="testing_yolo_2016_"+str(i),
#         folder = "testing_polygons_yolo_2016_30",  
#         fileFormat='TFRecord')
#     Task.start()
#     print(i)

In [40]:
# val_features = validation.toList(1467)
# for i in range(1467):
#     feature = ee.Feature(val_features.get(i))
#     fcp = val_points.filterBounds(feature.geometry())
#     train_db = neighbourhood_image.sampleRegions(collection=fcp, scale=30)
#     Task = ee.batch.Export.table.toDrive(
#         collection=train_db,        
#         description="validation_yolo_2016"+str(i),
#         fileNamePrefix="validation_yolo_2016_"+str(i),
#         folder = "validation_polygons_yolo_2016_30",  
#         fileFormat='TFRecord')
#     Task.start()
#     print(i)

# EXPORT

In [19]:
def export_2D_samples(collection, num_of_images, num_features, dimension, file_name_prefix, folder_name):
    filenames = []
    nbands = num_of_images*num_features+1
    nfeatures = dimension*dimension*nbands*len(collection) #estimate the totals # of features

    nparts = int(np.ceil(nfeatures/3e6))
    print('Dataset too long, splitting it into '+ str(nparts),'equal parts.')


    nppoints = np.array(collection)
    np.random.shuffle(nppoints)

    count_batch = 1  # Batch counter 

    for batch_arr in np.array_split(nppoints,nparts):

        fcp = ee.FeatureCollection([ee.Feature(ee.Geometry.Point(p),{'class':'NA'}) for p in batch_arr.tolist()])

        train_db = neighbourhood_image.sampleRegions(collection=fcp, scale=30)

        filename = file_name_prefix + str(count_batch)
        print('sending the task #%04d'%count_batch)
        Task = ee.batch.Export.table.toDrive(
                collection=train_db,        
                description=file_name_prefix+str(count_batch),
                fileNamePrefix=filename,
                folder = folder_name,  
                fileFormat='TFRecord')

        Task.start()
        filenames.append(filename)
        count_batch+=1

In [20]:
export_2D_samples(collection = training_pts, 
                  num_of_images = 9, 
                  num_features = 13, 
                  dimension = 7, 
                  file_name_prefix = "training_yolo_2016_", 
                  folder_name = "training_yolo_2016")

Dataset too long, splitting it into 44 equal parts.
sending the task #0001
sending the task #0002
sending the task #0003
sending the task #0004
sending the task #0005
sending the task #0006
sending the task #0007
sending the task #0008
sending the task #0009
sending the task #0010
sending the task #0011
sending the task #0012
sending the task #0013
sending the task #0014
sending the task #0015
sending the task #0016
sending the task #0017
sending the task #0018
sending the task #0019
sending the task #0020
sending the task #0021
sending the task #0022
sending the task #0023
sending the task #0024
sending the task #0025
sending the task #0026
sending the task #0027
sending the task #0028
sending the task #0029
sending the task #0030
sending the task #0031
sending the task #0032
sending the task #0033
sending the task #0034
sending the task #0035
sending the task #0036
sending the task #0037
sending the task #0038
sending the task #0039
sending the task #0040
sending the task #0041
sendi

In [21]:
export_2D_samples(collection = val_pts, 
                  num_of_images = 9, 
                  num_features = 13, 
                  dimension = 7, 
                  file_name_prefix = "validation_yolo_2016_", 
                  folder_name = "validation_yolo_2016")

Dataset too long, splitting it into 17 equal parts.
sending the task #0001
sending the task #0002
sending the task #0003
sending the task #0004
sending the task #0005
sending the task #0006
sending the task #0007
sending the task #0008
sending the task #0009
sending the task #0010
sending the task #0011
sending the task #0012
sending the task #0013
sending the task #0014
sending the task #0015
sending the task #0016
sending the task #0017


In [22]:
export_2D_samples(collection = testing_pts, 
                  num_of_images = 9, 
                  num_features = 13, 
                  dimension = 7, 
                  file_name_prefix = "testing_yolo_2016_", 
                  folder_name = "testing_yolo_2016")

Dataset too long, splitting it into 20 equal parts.
sending the task #0001
sending the task #0002
sending the task #0003
sending the task #0004
sending the task #0005
sending the task #0006
sending the task #0007
sending the task #0008
sending the task #0009
sending the task #0010
sending the task #0011
sending the task #0012
sending the task #0013
sending the task #0014
sending the task #0015
sending the task #0016
sending the task #0017
sending the task #0018
sending the task #0019
sending the task #0020
